# Detailed mission computation of the Pipistrel SW128

In [ ]:
import os.path as pth
import logging
import shutil

import fastoad.api as oad
import fastga_he.api as oad_he

DATA_FOLDER_PATH = "data"

WORK_FOLDER_PATH = "workdir"

CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, "pipistrel_mission_configuration.yml")
PT_FILE = pth.join(WORK_FOLDER_PATH, "pipistrel_assembly_detailed_mission.yml")
# The following PT file is not used for sizing just to try out the network function
SOURCE_FILE = pth.join(WORK_FOLDER_PATH, "pipistrel_mission_in.xml")

# For having log messages on screen
logging.basicConfig(level=logging.WARNING, format="%(levelname)-8s: %(message)s")

In [ ]:
# We copy all the useful file in the workdir

shutil.copy(pth.join(DATA_FOLDER_PATH, "pipistrel_mission_configuration.yml"), CONFIGURATION_FILE)
shutil.copy(pth.join(DATA_FOLDER_PATH, "pipistrel_assembly_detailed_mission.yml"), PT_FILE)
shutil.copy(pth.join(DATA_FOLDER_PATH, "pipistrel_mission_in.xml"), SOURCE_FILE)

In [ ]:
import warnings
import openmdao.api as om

warnings.filterwarnings("ignore", category=om.OpenMDAOWarning)

In [ ]:
NETWORK_FILE = pth.join(WORK_FOLDER_PATH, "pipistrel_assembly.html")

oad_he.power_train_network_viewer(power_train_file_path=PT_FILE, network_file_path=NETWORK_FILE)

from IPython.display import IFrame

# For some reason, this doesn't display icon. Opening it in Firefox does the trick.
IFrame(src=NETWORK_FILE, width="100%", height="500px")

In [ ]:
oad.list_modules(CONFIGURATION_FILE)

In [ ]:
oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

In [ ]:
import numpy as np

# To do the actual computation of the mission, we have to do some shenanigans to properly initiate the current outputed by the battery, else it doesn't converge.

configurator = oad.FASTOADProblemConfigurator(CONFIGURATION_FILE)
problem = configurator.get_problem(read_inputs=True)
problem.setup()

# An attempt to reach 48 kW in climb
# problem.set_val("data:mission:sizing:main_route:climb:climb_rate:sea_level", units="ft/min", val=550.0)
# problem.set_val("data:mission:sizing:main_route:climb:climb_rate:cruise_level", units="ft/min", val=510.0)

problem.set_val("data:mission:sizing:main_route:descent:descent_rate", units="ft/min", val=-500.0)

problem.run_model()
problem.write_outputs()

In [ ]:
MISSION_DATA_FILE = pth.join(WORK_FOLDER_PATH, "mission_data_detailed.csv")
PT_DATA_FILE = pth.join(WORK_FOLDER_PATH, "pipistrel_power_train_data.csv")

perfo_viewer = oad_he.PerformancesViewer(
    power_train_data_file_path=PT_DATA_FILE,
    mission_data_file_path=MISSION_DATA_FILE,
    plot_height=800,
)